# PIMA Indian Diabetes Dataset を使った糖尿病予測 - Google Colab版

このノートブックでは、PIMA Indian Diabetes Datasetを使用して、妊娠経験のある女性の身体測定データから糖尿病の有無を予測する機械学習モデルを構築します。

## データセットについて
- **対象**: ピマ族の妊娠経験のある女性（21歳以上）
- **特徴量**: 8つの身体測定値・検査値
- **目標変数**: 糖尿病の有無（0: なし、1: あり）
- **サンプル数**: 768件

## 特徴量の説明
1. **Pregnancies**: 妊娠回数
2. **Glucose**: 経口ブドウ糖負荷試験での血糖値
3. **BloodPressure**: 拡張期血圧（mm Hg）
4. **SkinThickness**: 上腕三頭筋の皮膚厚（mm）
5. **Insulin**: 2時間後の血清インスリン値（mu U/ml）
6. **BMI**: 体格指数（kg/m²）
7. **DiabetesPedigreeFunction**: 糖尿病系譜機能
8. **Age**: 年齢

## 使用する手法
- データ探索・可視化
- 前処理（欠損値処理、標準化）
- 複数のアルゴリズムでの学習・比較
- モデル評価・性能分析

## 1. 必要なライブラリのインストールとインポート

機械学習に必要なライブラリをインストール・インポートします。

In [ ]:
# 必要なライブラリをインストール
!pip install pandas numpy matplotlib seaborn scikit-learn

print("✅ 全てのライブラリのインストールが完了しました！")

In [ ]:
# データ処理・分析ライブラリ
import pandas as pd
import numpy as np

# 可視化ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns

# 機械学習ライブラリ
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

# その他
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'

# グラフの設定
plt.style.use('default')
sns.set_palette("husl")

print("✅ 全てのライブラリのインポートが完了しました！")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")
print(f"scikit-learn version: {sklearn.__version__}")

## 2. データセットのダウンロードと読み込み

PIMA Indian Diabetes Datasetをダウンロードして読み込みます。

In [ ]:
# データセットのダウンロード
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# カラム名を定義
column_names = [
    'Pregnancies',
    'Glucose', 
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI',
    'DiabetesPedigreeFunction',
    'Age',
    'Outcome'
]

# データを読み込み
print("📁 PIMA Indian Diabetes Datasetをダウンロード中...")
df = pd.read_csv(url, names=column_names)

print("✅ データセットの読み込みが完了しました！")
print(f"データの形状: {df.shape}")
print(f"特徴量数: {df.shape[1] - 1}")
print(f"サンプル数: {df.shape[0]}")

## 3. データの探索と可視化

データの基本統計量や分布を確認し、データの特性を理解します。

In [ ]:
# データの最初の5行を表示
print("📊 データの最初の5行:")
display(df.head())

# データの基本情報
print("\n📋 データの基本情報:")
print(df.info())

# 基本統計量
print("\n📈 基本統計量:")
display(df.describe())

In [ ]:
# 目標変数の分布
print("🎯 糖尿病の有無の分布:")
outcome_counts = df['Outcome'].value_counts()
print(f"糖尿病なし (0): {outcome_counts[0]}人 ({outcome_counts[0]/len(df)*100:.1f}%)")
print(f"糖尿病あり (1): {outcome_counts[1]}人 ({outcome_counts[1]/len(df)*100:.1f}%)")

# 目標変数の可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 棒グラフ
outcome_counts.plot(kind='bar', ax=axes[0], color=['skyblue', 'lightcoral'])
axes[0].set_title('糖尿病の有無の分布')
axes[0].set_xlabel('糖尿病')
axes[0].set_ylabel('人数')
axes[0].set_xticklabels(['なし', 'あり'], rotation=0)

# 円グラフ
axes[1].pie(outcome_counts.values, labels=['なし', 'あり'], autopct='%1.1f%%', 
           colors=['skyblue', 'lightcoral'], startangle=90)
axes[1].set_title('糖尿病の有無の割合')

plt.tight_layout()
plt.show()

In [ ]:
# 特徴量の分布を可視化
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()

feature_columns = df.columns[:-1]  # Outcome以外の列

for i, column in enumerate(feature_columns):
    # 糖尿病の有無による分布の違いを表示
    df[df['Outcome'] == 0][column].hist(alpha=0.7, bins=20, label='糖尿病なし', ax=axes[i], color='skyblue')
    df[df['Outcome'] == 1][column].hist(alpha=0.7, bins=20, label='糖尿病あり', ax=axes[i], color='lightcoral')
    
    axes[i].set_title(f'{column} の分布')
    axes[i].set_xlabel(column)
    axes[i].set_ylabel('頻度')
    axes[i].legend()

# 余ったsubplotを非表示
axes[8].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# 相関行列の可視化
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('特徴量間の相関行列')
plt.tight_layout()
plt.show()

# Outcomeとの相関が高い特徴量を表示
print("🔍 糖尿病の有無(Outcome)との相関:")
correlations = df.corr()['Outcome'].drop('Outcome').sort_values(key=abs, ascending=False)
for feature, corr in correlations.items():
    print(f"{feature:25}: {corr:6.3f}")

## 4. データの前処理

欠損値処理、異常値の確認、特徴量スケーリングを行います。

In [ ]:
# 欠損値の確認
print("❓ 欠損値の確認:")
missing_values = df.isnull().sum()
print(missing_values)

# 0値の確認（生物学的に0になりえない値）
print("\n🔍 0値の確認（生物学的に不可能な値）:")
zero_values = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for column in zero_values:
    zero_count = (df[column] == 0).sum()
    print(f"{column:20}: {zero_count:3d}個 ({zero_count/len(df)*100:5.1f}%)")

# データをコピーして前処理
df_processed = df.copy()

# 0値を中央値で置換（生物学的に0になりえない値のみ）
print("\n🔧 0値を中央値で置換中...")
for column in zero_values:
    if column in ['SkinThickness', 'Insulin']:  # 特に問題の多い列
        # 0以外の値の中央値で置換
        median_value = df_processed[df_processed[column] != 0][column].median()
        df_processed[column] = df_processed[column].replace(0, median_value)
        print(f"{column}: 0 → {median_value:.1f}")

print("✅ 前処理が完了しました！")

In [ ]:
# 特徴量とターゲットを分離
X = df_processed.drop('Outcome', axis=1)
y = df_processed['Outcome']

print(f"特徴量の形状: {X.shape}")
print(f"ターゲットの形状: {y.shape}")
print(f"特徴量名: {list(X.columns)}")

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 データ分割結果:")
print(f"訓練データ: {X_train.shape[0]}件")
print(f"テストデータ: {X_test.shape[0]}件")
print(f"訓練データの糖尿病率: {y_train.mean():.3f}")
print(f"テストデータの糖尿病率: {y_test.mean():.3f}")

# 特徴量の標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ データの分割と標準化が完了しました！")

## 5. 複数のモデルでの学習と比較

複数の機械学習アルゴリズムでモデルを学習し、性能を比較します。

In [ ]:
# 複数のモデルを定義
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'SVM': SVC(random_state=42, probability=True),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

# モデルの学習と評価
results = {}
print("🚀 複数のモデルで学習を開始...")

for name, model in models.items():
    print(f"\n📈 {name} を学習中...")
    
    # 交差検証でモデルを評価
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    
    # モデルを学習
    model.fit(X_train_scaled, y_train)
    
    # テストデータで予測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 性能指標を計算
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    results[name] = {
        'model': model,
        'cv_scores': cv_scores,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'test_accuracy': accuracy,
        'roc_auc': roc_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }
    
    print(f"交差検証精度: {cv_scores.mean():.3f} (±{cv_scores.std()*2:.3f})")
    print(f"テスト精度: {accuracy:.3f}")
    print(f"ROC AUC: {roc_auc:.3f}")

print("\n✅ 全てのモデルの学習が完了しました！")

In [ ]:
# モデル性能の比較表を作成
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'CV Accuracy': [results[name]['cv_mean'] for name in results.keys()],
    'CV Std': [results[name]['cv_std'] for name in results.keys()],
    'Test Accuracy': [results[name]['test_accuracy'] for name in results.keys()],
    'ROC AUC': [results[name]['roc_auc'] for name in results.keys()]
})

# ソートして表示
comparison_df = comparison_df.sort_values('Test Accuracy', ascending=False)
print("📊 モデル性能比較表:")
display(comparison_df)

# 最高性能のモデルを特定
best_model_name = comparison_df.iloc[0]['Model']
print(f"\n🏆 最高性能モデル: {best_model_name}")
print(f"テスト精度: {comparison_df.iloc[0]['Test Accuracy']:.3f}")
print(f"ROC AUC: {comparison_df.iloc[0]['ROC AUC']:.3f}")

## 6. モデル性能の詳細分析

最高性能のモデルについて詳細な分析を行います。

In [ ]:
# 最高性能モデルの詳細評価
best_model = results[best_model_name]['model']
best_y_pred = results[best_model_name]['y_pred']
best_y_pred_proba = results[best_model_name]['y_pred_proba']

# 分類レポート
print(f"📋 {best_model_name} の詳細評価:")
print("\n" + "="*50)
print("分類レポート:")
print(classification_report(y_test, best_y_pred, target_names=['糖尿病なし', '糖尿病あり']))

# 混同行列
cm = confusion_matrix(y_test, best_y_pred)
print("\n混同行列:")
print(cm)

# 混同行列の可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 混同行列
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=['糖尿病なし', '糖尿病あり'],
           yticklabels=['糖尿病なし', '糖尿病あり'], ax=axes[0])
axes[0].set_title(f'{best_model_name}\n混同行列')
axes[0].set_xlabel('予測値')
axes[0].set_ylabel('実際値')

# ROC曲線
fpr, tpr, thresholds = roc_curve(y_test, best_y_pred_proba)
roc_auc = roc_auc_score(y_test, best_y_pred_proba)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, 
            label=f'ROC curve (AUC = {roc_auc:.3f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('偽陽性率 (False Positive Rate)')
axes[1].set_ylabel('真陽性率 (True Positive Rate)')
axes[1].set_title(f'{best_model_name}\nROC曲線')
axes[1].legend(loc="lower right")
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 特徴量重要度の可視化（Random Forestの場合）
if best_model_name == 'Random Forest':
    # 特徴量重要度を取得
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # 可視化
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature', palette='viridis')
    plt.title(f'{best_model_name} - 特徴量重要度')
    plt.xlabel('重要度')
    plt.ylabel('特徴量')
    plt.tight_layout()
    plt.show()
    
    print("📊 特徴量重要度ランキング:")
    for i, (idx, row) in enumerate(feature_importance.iterrows(), 1):
        print(f"{i:2d}. {row['feature']:25}: {row['importance']:.4f}")

elif best_model_name == 'Logistic Regression':
    # 回帰係数を取得
    coefficients = pd.DataFrame({
        'feature': X.columns,
        'coefficient': best_model.coef_[0]
    })
    coefficients['abs_coefficient'] = np.abs(coefficients['coefficient'])
    coefficients = coefficients.sort_values('abs_coefficient', ascending=False)
    
    # 可視化
    plt.figure(figsize=(10, 6))
    colors = ['red' if x < 0 else 'blue' for x in coefficients['coefficient']]
    sns.barplot(data=coefficients, x='coefficient', y='feature', palette=colors)
    plt.title(f'{best_model_name} - 回帰係数')
    plt.xlabel('回帰係数')
    plt.ylabel('特徴量')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("📊 回帰係数ランキング（絶対値）:")
    for i, (idx, row) in enumerate(coefficients.iterrows(), 1):
        print(f"{i:2d}. {row['feature']:25}: {row['coefficient']:7.4f}")

## 7. 予測機能の実装とテスト

学習したモデルを使って新しいデータに対する予測を行う関数を作成し、実際にテストします。

In [ ]:
def predict_diabetes(pregnancies, glucose, blood_pressure, skin_thickness, 
                    insulin, bmi, diabetes_pedigree, age, model=best_model, scaler=scaler):
    """
    個人の身体測定データから糖尿病のリスクを予測する関数
    
    Parameters:
    - pregnancies: 妊娠回数
    - glucose: 血糖値
    - blood_pressure: 血圧
    - skin_thickness: 皮膚厚
    - insulin: インスリン値
    - bmi: BMI
    - diabetes_pedigree: 糖尿病系譜機能
    - age: 年齢
    
    Returns:
    - prediction: 予測結果 (0: 糖尿病なし, 1: 糖尿病あり)
    - probability: 糖尿病である確率
    """
    
    # 入力データを配列に変換
    input_data = np.array([[pregnancies, glucose, blood_pressure, skin_thickness,
                           insulin, bmi, diabetes_pedigree, age]])
    
    # 標準化
    input_data_scaled = scaler.transform(input_data)
    
    # 予測
    prediction = model.predict(input_data_scaled)[0]
    probability = model.predict_proba(input_data_scaled)[0][1]
    
    return prediction, probability

def interpret_prediction(prediction, probability):
    """
    予測結果を解釈しやすい形で表示する関数
    """
    print("🏥 糖尿病予測結果")
    print("=" * 30)
    
    if prediction == 1:
        print(f"⚠️  糖尿病のリスクが高いです")
        print(f"📊 糖尿病である確率: {probability:.1%}")
        
        if probability >= 0.8:
            risk_level = "非常に高い"
        elif probability >= 0.6:
            risk_level = "高い"
        else:
            risk_level = "中程度"
            
        print(f"🚨 リスクレベル: {risk_level}")
        print("💡 推奨事項: 医師の診察を受けることをお勧めします")
        
    else:
        print(f"✅ 糖尿病のリスクは低いです")
        print(f"📊 糖尿病である確率: {probability:.1%}")
        print(f"💡 推奨事項: 健康的な生活習慣を維持しましょう")

print("✅ 予測関数の準備が完了しました！")

In [ ]:
# テストケース1: 糖尿病リスクが高いケース
print("🧪 テストケース1: 高リスクケース")
print("患者情報: 35歳女性、妊娠歴6回、血糖値148、血圧72、皮膚厚35mm、インスリン0、BMI33.6、糖尿病系譜0.627")

prediction1, probability1 = predict_diabetes(
    pregnancies=6, glucose=148, blood_pressure=72, skin_thickness=35,
    insulin=0, bmi=33.6, diabetes_pedigree=0.627, age=50
)

interpret_prediction(prediction1, probability1)
print("\n" + "="*50 + "\n")

# テストケース2: 糖尿病リスクが低いケース
print("🧪 テストケース2: 低リスクケース")
print("患者情報: 25歳女性、妊娠歴1回、血糖値85、血圧66、皮膚厚29mm、インスリン0、BMI26.6、糖尿病系譜0.351")

prediction2, probability2 = predict_diabetes(
    pregnancies=1, glucose=85, blood_pressure=66, skin_thickness=29,
    insulin=0, bmi=26.6, diabetes_pedigree=0.351, age=31
)

interpret_prediction(prediction2, probability2)
print("\n" + "="*50 + "\n")

# テストケース3: 実際のテストデータから1件をランダムに選択
import random
random_idx = random.randint(0, len(X_test) - 1)
actual_outcome = y_test.iloc[random_idx]
test_data = X_test.iloc[random_idx]

print(f"🧪 テストケース3: 実際のテストデータ（インデックス: {random_idx}）")
print(f"実際の結果: {'糖尿病あり' if actual_outcome == 1 else '糖尿病なし'}")
print(f"患者データ: {dict(test_data)}")

prediction3, probability3 = predict_diabetes(
    pregnancies=test_data['Pregnancies'],
    glucose=test_data['Glucose'],
    blood_pressure=test_data['BloodPressure'],
    skin_thickness=test_data['SkinThickness'],
    insulin=test_data['Insulin'],
    bmi=test_data['BMI'],
    diabetes_pedigree=test_data['DiabetesPedigreeFunction'],
    age=test_data['Age']
)

interpret_prediction(prediction3, probability3)

# 予測の正確性をチェック
if prediction3 == actual_outcome:
    print("✅ 予測が正解しました！")
else:
    print("❌ 予測が外れました")

print(f"予測確率: {probability3:.3f}")

## 8. インタラクティブな予測ツール

ユーザーが数値を入力して糖尿病リスクを予測できるインタラクティブなツールを作成します。

In [ ]:
def interactive_diabetes_prediction():
    """
    インタラクティブな糖尿病予測ツール
    """
    print("🏥 糖尿病リスク予測ツール")
    print("=" * 40)
    print("以下の項目を入力してください：\n")
    
    try:
        # ユーザー入力
        pregnancies = int(input("妊娠回数: "))
        glucose = float(input("血糖値 (mg/dL): "))
        blood_pressure = float(input("拡張期血圧 (mmHg): "))
        skin_thickness = float(input("上腕三頭筋皮膚厚 (mm): "))
        insulin = float(input("2時間後血清インスリン値 (mu U/ml): "))
        bmi = float(input("BMI (kg/m²): "))
        diabetes_pedigree = float(input("糖尿病系譜機能: "))
        age = int(input("年齢: "))
        
        print("\n🔄 予測を実行中...")
        
        # 予測実行
        prediction, probability = predict_diabetes(
            pregnancies, glucose, blood_pressure, skin_thickness,
            insulin, bmi, diabetes_pedigree, age
        )
        
        print("\n")
        interpret_prediction(prediction, probability)
        
    except ValueError:
        print("❌ 正しい数値を入力してください")
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")

# 参考値の表示
print("📊 参考値：")
print("血糖値: 正常値 70-100 mg/dL")
print("血圧: 正常値 80-120 mmHg（拡張期）")
print("BMI: 正常値 18.5-24.9 kg/m²")
print("糖尿病系譜機能: 通常 0.0-2.0程度")
print("\n⚠️ 注意: この予測は参考値であり、医師の診断に代わるものではありません\n")

# インタラクティブツールの実行（コメントアウト - 必要に応じて有効化）
# interactive_diabetes_prediction()

## 9. モデルの保存と読み込み

学習したモデルを保存し、後で再利用できるようにします。

In [ ]:
import pickle
import joblib

# モデルとスケーラーを保存
print("💾 モデルとスケーラーを保存中...")

# 最高性能のモデルを保存
model_filename = f'diabetes_model_{best_model_name.lower().replace(" ", "_")}.pkl'
scaler_filename = 'diabetes_scaler.pkl'

# pickle を使用して保存
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)

with open(scaler_filename, 'wb') as f:
    pickle.dump(scaler, f)

print(f"✅ モデルを保存しました: {model_filename}")
print(f"✅ スケーラーを保存しました: {scaler_filename}")

# 保存したモデルを読み込んでテスト
print("\n📥 保存したモデルを読み込み中...")

with open(model_filename, 'rb') as f:
    loaded_model = pickle.load(f)

with open(scaler_filename, 'rb') as f:
    loaded_scaler = pickle.load(f)

# 読み込んだモデルでテスト予測
test_input = X_test.iloc[0:1]
test_scaled = loaded_scaler.transform(test_input)
loaded_prediction = loaded_model.predict(test_scaled)
original_prediction = best_model.predict(scaler.transform(test_input))

if loaded_prediction[0] == original_prediction[0]:
    print("✅ モデルの保存・読み込みが正常に完了しました！")
else:
    print("❌ モデルの保存・読み込みでエラーが発生しました")

print(f"元のモデル予測: {original_prediction[0]}")
print(f"読み込みモデル予測: {loaded_prediction[0]}")

## 🎯 まとめと結論

### 📊 プロジェクト成果

1. **データ分析**: PIMA Indian Diabetes Dataset (768サンプル) を詳細に分析
2. **前処理**: 欠損値処理、標準化、データ分割を実施
3. **モデル比較**: 4つの機械学習アルゴリズムを比較評価
4. **予測機能**: 実用的な糖尿病リスク予測システムを構築
5. **モデル保存**: 学習済みモデルの永続化を実現

### 🏆 最終結果

- **最高性能モデル**: {best_model_name}
- **テスト精度**: {results[best_model_name]['test_accuracy']:.3f}
- **ROC AUC**: {results[best_model_name]['roc_auc']:.3f}

### 🔍 重要な特徴量

糖尿病予測において特に重要な要因が特定されました。

### ⚠️ 注意事項

- この予測モデルは**医学的診断ツールではありません**
- あくまで**参考情報**として活用してください
- 実際の医療判断は**医師の診察**を受けてください
- モデルの性能は使用するデータセットに依存します

### 💡 今後の改善点

1. **より大規模なデータセット**での学習
2. **新しい特徴量**の追加（遺伝的要因、生活習慣など）
3. **アンサンブル手法**の活用
4. **クロスバリデーション**の更なる活用
5. **ハイパーパラメータ最適化**の実施

### 🔗 参考文献

- Smith, J.W., Everhart, J.E., Dickson, W.C., Knowler, W.C., & Johannes, R.S. (1988). Using the ADAP learning algorithm to forecast the onset of diabetes mellitus. In Proceedings of the Symposium on Computer Applications and Medical Care (pp. 261--265). IEEE Computer Society Press.

---

**🎉 お疲れ様でした！糖尿病予測モデルの構築が完了しました。**